In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import json

In [3]:
driver = webdriver.Chrome()

In [5]:
driver.get("https://room.chuo-u.ac.jp/ct/home")

In [6]:
driver.find_element(By.ID, "username_input").send_keys("23D7104001I")
driver.find_element(By.ID, "password_input").send_keys("1025t1025")
driver.find_element(By.ID, "login_button").click()
time.sleep(3)



In [18]:
driver.get("https://room.chuo-u.ac.jp/ct/home")
html = driver.page_source

In [20]:
soup = BeautifulSoup(html, "html.parser")
course_divs = soup.select("div.courselistweekly-c")

courses = []

for course_div in course_divs:
    link_tag = course_div.find("a", href=True)
    if not link_tag:
        continue

    name = link_tag.text.strip()
    link = link_tag["href"]

    fav_icon = course_div.select_one("a.courseweekly-fav img")
    is_favorite = not ("clip_off" in fav_icon["src"]) if fav_icon else False

    registration_state_tag = course_div.select_one("div.registration-state")
    registration_state = registration_state_tag.text.strip() if registration_state_tag else ""

    has_unsubmitted = any("未提出の課題があります" in img.get("alt", "") for img in course_div.find_all("img"))

    courses.append({
        "name": name,
        "link": link,
        "is_favorite": is_favorite,
        "registration_state": registration_state,
        "has_unsubmitted_assignments": has_unsubmitted
    })

In [24]:
html

'<html lang="ja" style="--header-menuv2-height: auto;"><head>\n <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n <meta http-equiv="cache-control" content="no-cache">\n  <meta http-equiv="Content-Script-Type" content="text/javascript">\n <meta http-equiv="pragma" content="no-cache">\n      <meta name="robots" content="NOINDEX, NOFOLLOW">\n   <meta name="google" content="notranslate">\n\n<meta name="viewport" content="">\n\n      <link rel="apple-touch-icon" href="/manaba-iconV2.png">\n       <link rel="shortcut icon" href="https://room.chuo-u.ac.jp/faviconV2.ico" type="image/x-icon">\n   \n   <link rel="stylesheet" href="/browser_std.css" type="text/css">\n  <link rel="stylesheet" href="/portal.css?1746002261" type="text/css">\n    <link rel="stylesheet" href="/fontsize_m.css" type="text/css">\n    <link rel="stylesheet" href="/color_1.css?1746002261" type="text/css">\n  <title>中央大学 manaba - home</title>\n <script src="/extjs/d3-3.5.11/d3.min.js" type="text/javascri

In [23]:
courses[2]

{'name': 'オペレーションズ・リサーチ１',
 'link': 'course_5646028',
 'is_favorite': False,
 'registration_state': '',
 'has_unsubmitted_assignments': False}

In [16]:
for index,el in enumerate(elements):
    course_name = el.text
    if "※リンク中" not in course_name:
        print(index,course_name)
    else:
        print(index,course_name,el.get_attribute("href"))

0 学問最前線
※リンク中 None
1 人工知能基礎論
2 オペレーションズ・リサーチ１
3 応用解析１
4 信頼性工学
5 機械学習基礎論
6 マーケティング・リサーチ
7 ヒューマンメディア工学
※リンク中 None
8 データサイエンス実験Ｂ
9 情報通信産業論
10 生命と多様性Ⅰ
11 ＯＲ演習
12 中国語ＢⅠ
13 コンピュータ工学基礎


In [ ]:

  # ログイン完了まで待つ（適宜調整）

# 3. 履修科目ページへアクセス
  # 例：ホームページに授業一覧がある場合

# 4. 授業一覧を取得

for el in elements:
    course_name = el.text
    course_link = el.get_attribute("href")
    courses.append({"name": course_name, "link": course_link})

# 5. JSONとして保存
with open("courses.json", "w", encoding="utf-8") as f:
    json.dump(courses, f, ensure_ascii=False, indent=2)

driver.quit()

In [25]:
def parse_timetable_from_manaba(html: str):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("table", class_="stdlist")

    # ヘッダーから曜日のリストを取得
    days = []
    for th in table.find("tr", class_="title").find_all("th")[1:]:  # 最初の1列目は時限列なので除く
        days.append(th.text.strip())

    timetable_data = []

    # 時限ごとの行を解析
    for tr in table.find_all("tr")[1:]:  # 最初の行はヘッダーなので除く
        tds = tr.find_all("td")
        if not tds:
            continue

        period = tds[0].text.strip()  # 時限番号

        # 各曜日の列を処理
        for i, td in enumerate(tds[1:]):  # 最初の列は時限列なので除く
            a_tag = td.find("a")
            if a_tag:
                title = a_tag.text.strip()
                href = a_tag.get("href")
                timetable_data.append({
                    "day": days[i],
                    "period": int(period) if period.isdigit() else period,
                    "title": title,
                    "href": href
                })

    return timetable_data

In [26]:
parse_timetable_from_manaba(html)

[{'day': '月', 'period': 1, 'title': '学問最前線', 'href': 'course_5644371'},
 {'day': '水', 'period': 1, 'title': '人工知能基礎論', 'href': 'course_5646094'},
 {'day': '月',
  'period': 2,
  'title': 'オペレーションズ・リサーチ１',
  'href': 'course_5646028'},
 {'day': '火', 'period': 2, 'title': '応用解析１', 'href': 'course_5645995'},
 {'day': '水', 'period': 2, 'title': '信頼性工学', 'href': 'course_5646055'},
 {'day': '木', 'period': 2, 'title': '機械学習基礎論', 'href': 'course_5646064'},
 {'day': '金', 'period': 2, 'title': 'マーケティング・リサーチ', 'href': 'course_5646043'},
 {'day': '月', 'period': 3, 'title': 'ヒューマンメディア工学', 'href': 'course_5646097'},
 {'day': '火', 'period': 3, 'title': 'データサイエンス実験Ｂ', 'href': 'course_5646010'},
 {'day': '木', 'period': 3, 'title': '情報通信産業論', 'href': 'course_5644755'},
 {'day': '月', 'period': 4, 'title': '生命と多様性Ⅰ', 'href': 'course_5643936'},
 {'day': '木', 'period': 4, 'title': 'ＯＲ演習', 'href': 'course_5646169'},
 {'day': '火', 'period': 5, 'title': '中国語ＢⅠ', 'href': 'course_5643408'},
 {'day': '水', 'period':